In [1]:
from llama_index import SimpleDirectoryReader, GPTVectorStoreIndex, \
                        LLMPredictor, PromptHelper, StorageContext, load_index_from_storage
from langchain import OpenAI

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [2]:
def index_documents(folder):
    max_input_size    = 4096
    num_outputs       = 512
    max_chunk_overlap = 20
    chunk_size_limit  = 600

    prompt_helper = PromptHelper(max_input_size, 
                                 num_outputs, 
                                 max_chunk_overlap, 
                                 chunk_size_limit = chunk_size_limit)
    
    llm_predictor = LLMPredictor(
        llm = OpenAI(temperature = 0, 
                         model_name = "text-davinci-003", 
                         max_tokens = num_outputs)
        )
#gpt-3.5-turbo"
    documents = SimpleDirectoryReader(folder).load_data()

    index = GPTVectorStoreIndex.from_documents(
                documents, 
                llm_predictor = llm_predictor, 
                prompt_helper = prompt_helper)

    index.storage_context.persist(persist_dir=".") # save in current directory

def load_index():
    # load the index from vector_store.json
    storage_context = StorageContext.from_defaults(persist_dir=".")
    index = load_index_from_storage(storage_context)
    return index
    
def my_GPT(input_text, index):
    # create a query engine to ask question
    query_engine = index.as_query_engine()
    response = query_engine.query(input_text)
    return response.response

In [3]:
index_documents("training_documents")

In [4]:
index = load_index()

In [5]:
while True:
    prompt = input("User: ")
    response = my_GPT(prompt, index)
    print(f'Agent: {response}\n')


User:  Act like an expert sparql developer and write a query for the following: which principal investigators have studied interventions for the condition COPD?


Agent: 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?PrincipalInvestigator
WHERE {
  ?Condition skos:prefLabel "COPD" .
  ?Intervention skos:prefLabel "treats" .
  ?Intervention rdfs:range ?Condition .
  ?PrincipalInvestigator skos:prefLabel "supervises" .
  ?PrincipalInvestigator rdfs:domain ?Intervention .
}



User:  Act like an expert sparql developer and write a query for the following: which principal investigators have studied interventions for the condition COPD?


Agent: 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?PrincipalInvestigator
WHERE {
  ?Condition skos:prefLabel "COPD" .
  ?Intervention skos:prefLabel "treats" .
  ?Intervention rdfs:range ?Condition .
  ?Study skos:prefLabel "supervises" .
  ?Study rdfs:domain ?Intervention .
  ?PrincipalInvestigator skos:prefLabel "submits protocol to" .
  ?PrincipalInvestigator rdfs:range ?Study



KeyboardInterrupt: Interrupted by user